In [1]:
import os
import time
import requests
from bs4 import BeautifulSoup
import gradio as gr
import openai
from dotenv import load_dotenv

from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Load environment variables
load_dotenv()

# OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Blogger OAuth client file
CLIENT_SECRET_FILE = "client_secret.json"  # Download from Google Cloud


In [2]:
SCOPES = ["https://www.googleapis.com/auth/blogger"]

def get_blogger_service():
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRET_FILE, SCOPES)
    creds = flow.run_local_server(port=0)  # Opens browser for OAuth
    service = build("blogger", "v3", credentials=creds)
    return service

def get_blog_id(service):
    """
    Fetches the first blog ID of the authenticated user.
    Raises an error if no blogs exist.
    """
    blogs = service.blogs().listByUser(userId="self").execute()
    if "items" not in blogs or not blogs["items"]:
        raise Exception("No blogs found. Please create a Blogger blog manually first.")
    blog = blogs["items"][0]
    return blog["id"], blog["name"]


In [3]:
def scrape_pinterest_profile(username):
    base_url = f"https://www.pinterest.com/{username}/"
    headers = {"User-Agent": "Mozilla/5.0"}

    r = requests.get(base_url, headers=headers)
    if r.status_code != 200:
        raise Exception("Profile not found or blocked. Make sure it's public.")
    soup = BeautifulSoup(r.text, "lxml")

    # Get board links
    board_links = []
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith(f"/{username}/") and href.count("/") == 2:
            board_links.append(href)
    board_links = list(dict.fromkeys(board_links))[:5]

    boards_data = []
    for link in board_links:
        board_name = link.split("/")[-2].replace("-", " ").title()
        full_board_url = f"https://www.pinterest.com{link}"
        try:
            pins = scrape_board_pins(full_board_url, headers)
            boards_data.append({
                "board_name": board_name,
                "pins": pins
            })
            time.sleep(1)
        except Exception:
            continue

    return {
        "username": username,
        "boards": boards_data,
        "bio": "Pinterest explorer with aesthetic obsessions and creative chaos."
    }

def scrape_board_pins(board_url, headers):
    soup = BeautifulSoup(requests.get(board_url, headers=headers).text, "lxml")
    pin_texts = []
    for tag in soup.find_all(["h3", "img", "div"]):
        text = ""
        if tag.name == "h3":
            text = tag.get_text(strip=True)
        elif tag.name == "img":
            text = tag.get("alt", "")
        elif tag.name == "div":
            text = tag.get_text(strip=True)
        if text and 3 < len(text) < 120:
            pin_texts.append(text)
    return list(dict.fromkeys(pin_texts))[:8]


In [4]:
def generate_blog(profile_data, tone="Funny"):
    board_descriptions = ""
    for b in profile_data["boards"]:
        pins_sample = ", ".join(b["pins"])
        board_descriptions += f"- {b['board_name']}: {pins_sample}\n"

    prompt = f"""
You are a witty, observant blogger.

Pinterest profile data:
Username: {profile_data['username']}
Bio: {profile_data['bio']}

Boards and sample pins:
{board_descriptions}

Write a {tone.lower()} blog (600–800 words) titled:
"Inside the Mind of a Pinterest Dreamer"

- Use humor, irony, and self-awareness.
- Include subheadings.
- End with a funny, reflective conclusion.
"""

    client = openai.OpenAI()
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.8,
        max_tokens=1200
    )
    return response.choices[0].message.content


In [5]:
def publish_to_blogger(blog_id, title, content, labels=None):
    service = get_blogger_service()
    post_body = {
        "kind": "blogger#post",
        "blog": {"id": blog_id},
        "title": title,
        "content": content.replace("\n", "<br>"),
        "labels": labels or []
    }
    post = service.posts().insert(blogId=blog_id, body=post_body, isDraft=False).execute()
    return post["url"]


In [6]:
def create_blog_and_publish(pinterest_username, tone="Funny"):
    try:
        # Scrape Pinterest
        profile_data = scrape_pinterest_profile(pinterest_username)
        if not profile_data["boards"]:
            return "⚠️ No boards or pins found (maybe a private account?)."

        # Generate blog
        blog_text = generate_blog(profile_data, tone)

        # Blogger auth & fetch existing blog
        service = get_blogger_service()
        blog_id, blog_name = get_blog_id(service)

        # Publish post
        url = publish_to_blogger(blog_id, f"Inside the Mind of {pinterest_username} on Pinterest", blog_text, labels=["Pinterest", "Humor"])

        return f"✅ Blog published at: {url}"

    except Exception as e:
        return f"❌ Error occurred: {type(e).__name__} - {str(e)}"


In [7]:
app = gr.Interface(
    fn=create_blog_and_publish,
    inputs=[
        gr.Textbox(label="Pinterest Username", placeholder="e.g. artandmatcha"),
        gr.Radio(["Funny", "Poetic", "Sarcastic"], label="Tone", value="Funny")
    ],
    outputs=gr.Textbox(label="Status"),
    title="🎨 Pinterest → GPT-4o-mini → Blogger Auto Publisher",
    description="Scrapes a Pinterest profile, generates a humorous blog, and publishes to Blogger automatically. Requires a manually created blog."
)

app.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
